In [1]:
import boto3, json

In [2]:
bedrock_runtime = boto3.client(
    service_name='bedrock-runtime',
    region_name='us-east-1',
)

In [3]:
prompt = "Hello"
json_data = "[{'name': '.dockerignore'}, {'name': '.env.example'}, {'name': '.github'}, {'name': '.gitignore'}, {'name': '.pre-commit-config.yaml'}, {'name': 'LICENSE.md'}, {'name': 'README.md'}, {'name': 'backend'}, {'name': 'codecov.yaml'}, {'name': 'docker-compose.yaml'}]"

In [4]:
kwargs = {
  "modelId": "cohere.command-text-v14",
  "contentType": "application/json",
  "accept": "*/*",
  "body": "{\"prompt\":\"Act as a bot. Explain this github repository by a key-pair ({'name': ..., 'description'}) value. You just need to explain the potential content in each folder. Return your answer in JSON format without any further explanation. Provide your answer in ```json ```.\\n\\n" + json_data + "\",\"max_tokens\":4000,\"temperature\":0.75,\"p\":0.35,\"k\":128,\"stop_sequences\":[],\"return_likelihoods\":\"NONE\"}",
}

print(kwargs)

{'modelId': 'cohere.command-text-v14', 'contentType': 'application/json', 'accept': '*/*', 'body': '{"prompt":"Act as a bot. Explain this github repository by a key-pair ({\'name\': ..., \'description\'}) value. You just need to explain the potential content in each folder. Return your answer in JSON format without any further explanation. Provide your answer in ```json ```.\\n\\n[{\'name\': \'.dockerignore\'}, {\'name\': \'.env.example\'}, {\'name\': \'.github\'}, {\'name\': \'.gitignore\'}, {\'name\': \'.pre-commit-config.yaml\'}, {\'name\': \'LICENSE.md\'}, {\'name\': \'README.md\'}, {\'name\': \'backend\'}, {\'name\': \'codecov.yaml\'}, {\'name\': \'docker-compose.yaml\'}]","max_tokens":4000,"temperature":0.75,"p":0.35,"k":128,"stop_sequences":[],"return_likelihoods":"NONE"}'}


In [5]:
response = bedrock_runtime.invoke_model(**kwargs)

print(response)

{'ResponseMetadata': {'RequestId': '98e41727-eaa5-4f21-ad0a-d4eba71d22b7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Sun, 05 Nov 2023 03:09:35 GMT', 'content-type': 'application/json', 'content-length': '3508', 'connection': 'keep-alive', 'x-amzn-requestid': '98e41727-eaa5-4f21-ad0a-d4eba71d22b7'}, 'RetryAttempts': 0}, 'contentType': 'application/json', 'body': <botocore.response.StreamingBody object at 0x00000120ABF93700>}


In [6]:
response_body = json.loads(response['body'].read()) 

print(response_body)

{'generations': [{'finish_reason': 'COMPLETE', 'id': '3dfbf358-774d-49c1-9a91-22720a134a75', 'text': ' ```json\n[\n  {\n    "name": ".dockerignore",\n    "description": "A list of files or directories to ignore, preventing them from being included in the build context when using Docker. You can specify ignore rules such as ignoring all .env files or temporary build artifacts."\n  },\n  {\n    "name": ".env.example",\n    "description": "An example of a .env file, which often contains environment variables required for your application to run correctly. This file can be used as a reference to help developers set up their local environment."\n  },\n  {\n    "name": ".github",\n    "description": ".github directory that contains files related to GitHub integration and workflows. It often includes templates for issue and pull request templates, code owners file, and guidelines for the development process."\n  },\n  {\n    "name": ".gitignore",\n    "description": "A Git ignore file that sp

In [7]:
data = response_body['generations'][0]['text']

In [8]:
import re

json_data = re.search(r'```json(.*?)```', data, re.DOTALL)

if json_data:
    # Extracting the JSON content from the matched pattern
    json_content = json_data.group(1)
    
    # Converting the extracted JSON content into a Python dictionary
    data_dict = json.loads(json_content)
else:
    print("No JSON data found between ```json and ```.")

print(type(data_dict))
data_dict

<class 'list'>


[{'name': '.dockerignore',
  'description': 'A list of files or directories to ignore, preventing them from being included in the build context when using Docker. You can specify ignore rules such as ignoring all .env files or temporary build artifacts.'},
 {'name': '.env.example',
  'description': 'An example of a .env file, which often contains environment variables required for your application to run correctly. This file can be used as a reference to help developers set up their local environment.'},
 {'name': '.github',
  'description': '.github directory that contains files related to GitHub integration and workflows. It often includes templates for issue and pull request templates, code owners file, and guidelines for the development process.'},
 {'name': '.gitignore',
  'description': 'A Git ignore file that specifies rules for files and directories to ignore, preventing them from being tracked by Git. This can be used to exclude temporary build artifacts, compiled code, or per